<a href="https://colab.research.google.com/github/codesxas/MLDM-Assignments/blob/master/Assignment_9/MLDM_lab_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Problem 1

In [0]:
def loadData (file_path):
  df = pd.read_csv(file_path)
  y = df['5'].to_numpy()
  df.drop(columns='5', inplace=True)
  x = df.to_numpy()

  return x, y


In [3]:
x, y = loadData('mnist_train.csv')
print(y.shape)
print(x.shape)

(9999,)
(9999, 784)


In [0]:
x_train,x_test,y_train,y_test=train_test_split(x, y, test_size=0.25,random_state=42)

Labels = pd.get_dummies(y_train)

Problem 2

In [0]:
class Perceptron():
    def __init__(self,x,y):
        self.x = x/255   # Divide by 255 to normalise the pixel values (0-255)
        self.y = y
        self.weights = []
        self.bias = []
        self.outputs = []
        self.derivatives = []
        self.activations = []
        
    def connect(self, layer1, layer2):
        #Initialise weights,derivatives and activation lists
        self.derivatives.append(np.random.uniform(0,0.1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.weights.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.bias.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))


    def softmax(self, z):
        e=np.exp(z)
        return e/np.sum(e, axis=1).reshape(-1, 1) 
    
    def max_log_likelihood(self, y_pred, y):
        return y*np.log(y_pred)
    
    def delta_mll(self, y, y_pred):
        return y_pred-y
    
    def forward_pass(self, x, y, weights, bias):
        cost=0
        self.outputs=[]
        for i in range(len(weights)):
            samples = len(x)
            ones_array = np.ones(samples).reshape(samples, 1)
            self.outputs.append(x) #append without adding ones array
            z = np.dot(np.append(ones_array, x, axis=1),weights[i]+bias[i])
            x = self.softmax(z)
        self.outputs.append(x)
        self.y_pred = x
        temp = -self.max_log_likelihood(self.y_pred, y)
        cost = np.mean(np.sum(temp,axis=1))
        return cost
    
    
    def backward_pass(self,y,lr):
        for i in range(len(self.weights)-1,-1,-1):
            ones_array = np.ones(len(neuron.outputs[i])).reshape(len(neuron.outputs[i]), 1)
            prev_term = self.delta_mll(y, self.y_pred)  
            # derivatives follow specific order,last three terms added new,rest from previous term  
            self.derivatives[i]=np.dot(prev_term.T,np.append(ones_array,self.outputs[i],axis=1))   
            self.weights[i]=self.weights[i]-lr*((self.derivatives[i].T)/len(y))
            self.bias[i]=self.bias[i]-lr*((self.derivatives[i].T)/len(y))
    
    def train(self,batches,lr=1e-3,epoch=10):
        """number of batches to split data in,Learning rate and epochs"""
        for epochs in range(epoch):
            samples=len(self.x)
            c=0
            for i in range(batches):
              x_batch=self.x[int((samples/batches)*i):int((samples/batches)*(i+1))]
              y_batch=self.y.loc[int((samples/batches)*i):int((samples/batches)*(i+1))-1]
              
              c=self.forward_pass(x_batch,y_batch,self.weights,self.bias)
              self.backward_pass(y_batch,lr)
            print(epochs,c/batches)
    
    def predict(self,x):
        """input: x_test values"""
        x=x/255
        for i in range(len(self.weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            z=np.dot(np.append(ones_array,x,axis=1),self.weights[i]+self.bias[i])
            x=self.softmax(z) 
        return np.argmax(x,axis=1)

In [14]:
neuron = Perceptron(x_train,Labels)
neuron.connect(x_train, Labels)
neuron.train(batches=1000, lr=0.2, epoch=30)

0 0.001576014225144764
1 0.0011333198187482582
2 0.0012372264078219714
3 0.0013989832011237495
4 0.0014262177775442157
5 0.0012822252459975473
6 0.0012285712114221256
7 0.001209359039257535
8 0.0011668642019784412
9 0.0011518753435283144
10 0.0010590585679035832
11 0.0010239267975151115
12 0.000966569459911783
13 0.0008983535075969271
14 0.0008224324622228703
15 0.0007494184830235992
16 0.0006823447543905542
17 0.0006162499323402363
18 0.0005459317630848759
19 0.0004736235437849922
20 0.00040457737580726897
21 0.00034076747001493166
22 0.0002831053786427962
23 0.00023317215338180478
24 0.00019309581231568578
25 0.00016456854945019248
26 0.0001464674092700145
27 0.00013591192028107992
28 0.00012991245106966186
29 0.00012578340787275168


In [15]:
pred = neuron.predict(x_test)
np.bincount(neuron.predict(x_test)),np.bincount(y_test)

(array([267, 278, 265, 277, 272, 242, 236, 264, 160, 239]),
 array([256, 268, 262, 271, 266, 232, 231, 276, 206, 232]))

In [20]:
print(f"accuracy is {np.bincount(np.abs(y_test-pred))[0]*100/len(y_test)} %")

accuracy is 88.92 %


Problem 3

In [0]:
class Layer():
    def __init__(self,size,activation='sigmoid'): 
        self.shape=(1,size)
        self.activation=activation
                
class SingleLayerNeuralNetwork():
    def __init__(self,x,y):

        self.x=x/255   # Divide by 255 to normalise the pixel values (0-255)
        self.y=y
        self.weights=[]
        self.bias=[]
        self.outputs=[]
        self.derivatives=[]
        self.activations=[]
        
    def connect(self,layer1,layer2):
        #Initialise weights,derivatives and activation lists
        self.derivatives.append(np.random.uniform(0,0.1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.weights.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.bias.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))
        if isinstance(layer2,Layer):
            self.activations.append(layer2.activation)
            
    def activation(self,name,z,derivative=False):
        
        #implementation of various activation functions and their derivatives
        if name=='sigmoid':
            if derivative==False:
                return 1/(1+np.exp(-z))
            else:
                return z*(1-z)
        
    def softmax(self,z):
        e=np.exp(z)
        return e/np.sum(e,axis=1).reshape(-1,1) 
    
    def max_log_likelihood(self,y_pred,y):
        return y*np.log(y_pred)
    
    def delta_mll(self,y,y_pred):
        return y_pred-y
    
    def forward_pass(self,x,y,weights,bias):
        cost=0
        self.outputs=[]
        for i in range(len(weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            self.outputs.append(x) #append without adding ones array
            z=np.dot(np.append(ones_array,x,axis=1),weights[i]+bias[i])
            if i==len(weights)-1:
                x=self.softmax(z)
            else:
                x=self.activation(self.activations[i],z)
        self.outputs.append(x)
        self.y_pred=x
        
        temp=-self.max_log_likelihood(self.y_pred,y)
        cost=np.mean(np.sum(temp,axis=1))
        return cost
    
    
    def backward_pass(self,y,lr):
        for i in range(len(self.weights)-1,-1,-1):
            ones_array=np.ones(len(self.outputs[i])).reshape(len(self.outputs[i]),1)
            if i==len(self.weights)-1:
                prev_term=self.delta_mll(y,self.y_pred)  
                # derivatives follow specific order,last three terms added new,rest from previous term  
                self.derivatives[i]=np.dot(prev_term.T,np.append(ones_array,self.outputs[i],axis=1))   
            else:
                prev_term=np.dot(prev_term,self.weights[i+1][1:].T)*self.activation(self.activations[i],self.outputs[i+1],derivative=True)
                self.derivatives[i]=np.dot(prev_term.T,np.append(ones_array,self.outputs[i],axis=1))
            self.weights[i]=self.weights[i]-lr*((self.derivatives[i].T)/len(y))
            self.bias[i]=self.bias[i]-lr*((self.derivatives[i].T)/len(y))
                
    
    def train(self,batches,lr=1e-3,epoch=10):
        for epochs in range(epoch):
            samples=len(self.x)
            c=0
            for i in range(batches):
              x_batch=self.x[int((samples/batches)*i):int((samples/batches)*(i+1))]
              y_batch=self.y.loc[int((samples/batches)*i):int((samples/batches)*(i+1))-1]
              
              c=self.forward_pass(x_batch,y_batch,self.weights,self.bias)
              self.backward_pass(y_batch,lr)
            print(epochs,c/batches)
    
    def predict(self,x):
        x=x/255
        for i in range(len(self.weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            z=np.dot(np.append(ones_array,x,axis=1),self.weights[i]+self.bias[i])
            if i==len(self.weights)-1:
                x=self.softmax(z)
            else:
                x=self.activation(self.activations[i],z)

        return np.argmax(x,axis=1)

In [24]:
neuron = SingleLayerNeuralNetwork(x_train, Labels)
layer = Layer(100)
neuron.connect(x_train, layer)
neuron.connect(layer, Labels)
neuron.train(batches=1000,lr=0.1,epoch=50)

0 0.00028759624659275326
1 0.00032420847030328426
2 0.00039098816005103367
3 0.00042494329662581457
4 0.0004098569547816444
5 0.0003617075379566328
6 0.00029738642332423565
7 0.00024350285311956275
8 0.00019415866781650375
9 0.00015385196003813996
10 0.00012303890905348316
11 9.814271092557557e-05
12 7.845237405599915e-05
13 6.400396913720192e-05
14 5.3491145206895864e-05
15 4.5471626140973745e-05
16 3.9121583383108105e-05
17 3.3973941353526116e-05
18 2.973257336724531e-05
19 2.6217511571827444e-05
20 2.3312583101697508e-05
21 2.0929133659263635e-05
22 1.8986838101740678e-05
23 1.7407047292848282e-05
24 1.610956433915365e-05
25 1.5016055043572032e-05
26 1.4069854425211783e-05
27 1.3245747763101272e-05
28 1.2533027613417771e-05
29 1.1910148658985198e-05
30 1.1356024744385212e-05
31 1.0855711284144597e-05
32 1.039866976409461e-05
33 9.977346674632008e-06
34 9.586272105021964e-06
35 9.221406555216508e-06
36 8.87967951646072e-06
37 8.55868166278962e-06
38 8.256465015571151e-06
39 7.9714129

In [29]:
pred=neuron.predict(x_test)
np.bincount(neuron.predict(x_test)),np.bincount(y_test)

(array([256, 268, 262, 273, 260, 227, 238, 275, 205, 236]),
 array([256, 268, 262, 271, 266, 232, 231, 276, 206, 232]))

In [30]:
print(f"accuracy is {np.bincount(np.abs(y_test-pred))[0]*100/len(y_test)} %")

accuracy is 92.4 %


Problem 4

In [0]:
class Layer():
    def __init__(self,size,activation='sigmoid'): 
        self.shape=(1,size)
        self.activation=activation
                
class DoubleLayerNeuralNetwork():
    def __init__(self,x,y):
        self.x=x/255   # Divide by 255 to normalise the pixel values (0-255)
        self.y=y
        self.weights=[]
        self.bias=[]
        self.outputs=[]
        self.derivatives=[]
        self.activations=[]
        
    def connect(self,layer1,layer2):
        #Initialise weights,derivatives and activation lists
        self.derivatives.append(np.random.uniform(0,0.1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.weights.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.bias.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))
        if isinstance(layer2,Layer):
            self.activations.append(layer2.activation)
            
    def activation(self,name,z,derivative=False):
        #implementation of various activation functions and their derivatives
        if name=='sigmoid':
            if derivative==False:
                return 1/(1+np.exp(-z))
            else:
                return z*(1-z)
        
    def softmax(self,z):
        e=np.exp(z)
        return e/np.sum(e,axis=1).reshape(-1,1) 
    
    def max_log_likelihood(self,y_pred,y):
        return y*np.log(y_pred)
    
    def delta_mll(self,y,y_pred):
        return y_pred-y
    
    def forward_pass(self,x,y,weights,bias):
        cost=0
        self.outputs=[]
        for i in range(len(weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            self.outputs.append(x) #append without adding ones array
            z=np.dot(np.append(ones_array,x,axis=1),weights[i]+bias[i])
            if i==len(weights)-1:
                x=self.softmax(z)
            else:
                x=self.activation(self.activations[i],z)
        self.outputs.append(x)
        self.y_pred=x
        
        temp=-self.max_log_likelihood(self.y_pred,y)
        cost=np.mean(np.sum(temp,axis=1))
        return cost
    
    
    def backward_pass(self,y,lr):
        for i in range(len(self.weights)-1,-1,-1):
            ones_array=np.ones(len(self.outputs[i])).reshape(len(self.outputs[i]),1)
            if i==len(self.weights)-1:
                prev_term=self.delta_mll(y,self.y_pred)  
                # derivatives follow specific order,last three terms added new,rest from previous term  
                self.derivatives[i]=np.dot(prev_term.T,np.append(ones_array,self.outputs[i],axis=1))   
            else:
                prev_term=np.dot(prev_term,self.weights[i+1][1:].T)*self.activation(self.activations[i],self.outputs[i+1],derivative=True)
                self.derivatives[i]=np.dot(prev_term.T,np.append(ones_array,self.outputs[i],axis=1))
            self.weights[i]=self.weights[i]-lr*((self.derivatives[i].T)/len(y))
            self.bias[i]=self.bias[i]-lr*((self.derivatives[i].T)/len(y))
                
    
    def train(self,batches,lr=1e-3,epoch=10):
        for epochs in range(epoch):
            samples=len(self.x)
            c=0
            for i in range(batches):
              x_batch=self.x[int((samples/batches)*i):int((samples/batches)*(i+1))]
              y_batch=self.y.loc[int((samples/batches)*i):int((samples/batches)*(i+1))-1]
              
              c=self.forward_pass(x_batch,y_batch,self.weights,self.bias)
              self.backward_pass(y_batch,lr)
            print(epochs,c/batches)
    
    def predict(self,x):
        x=x/255
        for i in range(len(self.weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            z=np.dot(np.append(ones_array,x,axis=1),self.weights[i]+self.bias[i])
            if i==len(self.weights)-1:
                x=self.softmax(z)
            else:
                x=self.activation(self.activations[i],z)
        return np.argmax(x,axis=1)

In [32]:
neuron = DoubleLayerNeuralNetwork(x_train, Labels)
layer1 = Layer(100)
layer2 = Layer(100)
neuron.connect(x_train, layer1)
neuron.connect(layer1, layer2)
neuron.connect(layer2, Labels)
neuron.train(batches=1000, lr=0.1, epoch=20)

0 0.0009246753650025163
1 0.0006890180770856735
2 0.0007660625429046436
3 0.0007974766193253802
4 0.0007940729326666774
5 0.0007863715120282105
6 0.0007627120251324064
7 0.0007231438994891776
8 0.0006670903351895808
9 0.00060296516834619
10 0.0005379537057706004
11 0.00046747929709260944
12 0.00038790942945181545
13 0.00030298736969095613
14 0.00022530915977293195
15 0.00016738294062619213
16 0.00013021973476943314
17 0.00010630107279707621
18 8.952215006901322e-05
19 7.670710813667534e-05


In [33]:
pred = neuron.predict(x_test)
np.bincount(neuron.predict(x_test)),np.bincount(y_test)

(array([269, 264, 261, 265, 269, 211, 248, 277, 198, 238]),
 array([256, 268, 262, 271, 266, 232, 231, 276, 206, 232]))

In [35]:
print(f"accuracy is {np.bincount(np.abs(y_test-pred))[0]*100/len(y_test)} %")

accuracy is 90.64 %


Problem 5

In [0]:
class Layer():
    def __init__(self,size,activation='sigmoid'): 
        self.shape=(1,size)
        self.activation=activation
                
class NeuralNetworkActivations():
    def __init__(self,x,y):
        self.x=x/255   # Divide by 255 to normalise the pixel values (0-255)
        self.y=y
        self.weights=[]
        self.bias=[]
        self.outputs=[]
        self.derivatives=[]
        self.activations=[]
        
    def connect(self,layer1,layer2):
        #Initialise weights,derivatives and activation lists
        self.derivatives.append(np.random.uniform(0,0.1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.weights.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.bias.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))
        if isinstance(layer2,Layer):
            self.activations.append(layer2.activation)
            
    def activation(self,name,z,derivative=False):
        #implementation of various activation functions and their derivatives
        if name=='sigmoid':
            if derivative==False:
                return 1/(1+np.exp(-z))
            else:
                return z*(1-z)
        elif name=='relu':
            if derivative==False:
                return np.maximum(0.0,z)
            else:
              z[z<=0] = 0.0
              z[z>0] = 1.0
              return z
        elif name=='tanh':
          if derivative==False:
                return np.tanh(z)
          else:
                return 1.0 - (np.tanh(z)) ** 2
        
    def softmax(self,z):
        e=np.exp(z)
        return e/np.sum(e,axis=1).reshape(-1,1) 
    
    def max_log_likelihood(self,y_pred,y):
        return y*np.log(y_pred)
    
    def delta_mll(self,y,y_pred):
        return y_pred-y
    
    def forward_pass(self,x,y,weights,bias):
        cost=0
        self.outputs=[]
        for i in range(len(weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            self.outputs.append(x) #append without adding ones array
            z=np.dot(np.append(ones_array,x,axis=1),weights[i]+bias[i])
            if i==len(weights)-1:
                x=self.softmax(z)
            else:
                x=self.activation(self.activations[i],z)
        self.outputs.append(x)
        self.y_pred=x
        
        temp=-self.max_log_likelihood(self.y_pred,y)
        cost=np.mean(np.sum(temp,axis=1))
        return cost
    
    
    def backward_pass(self,y,lr):
        for i in range(len(self.weights)-1,-1,-1):
            ones_array=np.ones(len(self.outputs[i])).reshape(len(self.outputs[i]),1)
            if i==len(self.weights)-1:
                prev_term=self.delta_mll(y,self.y_pred)  
                # derivatives follow specific order,last three terms added new,rest from previous term  
                self.derivatives[i]=np.dot(prev_term.T,np.append(ones_array,self.outputs[i],axis=1))   
            else:
                prev_term=np.dot(prev_term,self.weights[i+1][1:].T)*self.activation(self.activations[i],self.outputs[i+1],derivative=True)
                self.derivatives[i]=np.dot(prev_term.T,np.append(ones_array,self.outputs[i],axis=1))
            self.weights[i]=self.weights[i]-lr*((self.derivatives[i].T)/len(y))
            self.bias[i]=self.bias[i]-lr*((self.derivatives[i].T)/len(y))
    
    def train(self,batches,lr=1e-3,epoch=10):
        for epochs in range(epoch):
            samples=len(self.x)
            c=0
            for i in range(batches):
              x_batch=self.x[int((samples/batches)*i):int((samples/batches)*(i+1))]
              y_batch=self.y.loc[int((samples/batches)*i):int((samples/batches)*(i+1))-1]
              
              c=self.forward_pass(x_batch,y_batch,self.weights,self.bias)
              self.backward_pass(y_batch,lr)
            print(epochs,c/batches)
    
    def predict(self,x):
        x=x/255
        for i in range(len(self.weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            z=np.dot(np.append(ones_array,x,axis=1),self.weights[i]+self.bias[i])
            if i==len(self.weights)-1:
                x=self.softmax(z)
            else:
                x=self.activation(self.activations[i],z)
        return np.argmax(x,axis=1)

In [36]:
neuron=NeuralNetworkActivations(x_train,Labels)
layer1 = Layer(100, 'sigmoid')
layer2 = Layer(50, 'tanh')
neuron.connect(x_train, layer1)
neuron.connect(layer1, layer2)
neuron.connect(layer2, Labels)
neuron.train(batches=1000, lr=0.1, epoch=20)

0 0.00022902306486852664
1 0.00016994320112346742
2 9.380919530023971e-05
3 0.00028509603182765887
4 0.0001983337955970099
5 7.080909251828251e-05
6 0.00041963409453966614
7 3.795911883176497e-05
8 3.0138572660946493e-05
9 1.35979999587421e-05
10 2.1028787768211475e-05
11 9.358667349194094e-05
12 6.399837832880523e-06
13 5.412895269344516e-06
14 1.5795681504797766e-06
15 1.1282692025572214e-05
16 0.00011007114581353774
17 1.0964291650900242e-06
18 2.562533901532028e-06
19 2.860794166002153e-06


In [37]:
pred = neuron.predict(x_test)
np.bincount(neuron.predict(x_test)), np.bincount(y_test)

(array([262, 261, 244, 290, 272, 220, 235, 269, 208, 239]),
 array([256, 268, 262, 271, 266, 232, 231, 276, 206, 232]))

In [38]:
print(f"accuracy is {np.bincount(np.abs(y_test-pred))[0]*100/len(y_test)} %")

accuracy is 92.48 %


Problem 6

In [0]:
class Layer():
    def __init__(self,size,activation='sigmoid'): 
        self.shape=(1,size)
        self.activation=activation
                
class NeuralNetworkMomentum():
    def __init__(self,x,y):
        self.x=x/255   # Divide by 255 to normalise the pixel values (0-255)
        self.y=y
        self.weights=[]
        self.bias=[]
        self.outputs=[]
        self.derivatives=[]
        self.activations=[]
        self.delta_weights=[]
        self.delta_bias=[]
        
    def connect(self,layer1,layer2):
        #Initialise weights,derivatives and activation lists
        self.derivatives.append(np.random.uniform(0,0.1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.weights.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.bias.append(np.random.uniform(-1,1,size=(layer1.shape[1]+1,layer2.shape[1])))
        self.delta_weights.append(np.zeros((layer1.shape[1]+1,layer2.shape[1])))
        self.delta_bias.append(np.zeros((layer1.shape[1]+1,layer2.shape[1])))
        if isinstance(layer2,Layer):
            self.activations.append(layer2.activation)
            
    def activation(self,name,z,derivative=False):
        #implementation of various activation functions and their derivatives
        if name=='sigmoid':
            if derivative==False:
                return 1/(1+np.exp(-z))
            else:
                return z*(1-z)
        elif name=='relu':
            if derivative==False:
                return np.maximum(0.0,z)
            else:
              z[z<=0] = 0.0
              z[z>0] = 1.0
              return z
        elif name=='tanh':
          if derivative==False:
                return np.tanh(z)
          else:
                return 1.0 - (np.tanh(z)) ** 2
        
    def softmax(self,z):
        e=np.exp(z)
        return e/np.sum(e,axis=1).reshape(-1,1) 
    
    def max_log_likelihood(self,y_pred,y):
        return y*np.log(y_pred)
    
    def delta_mll(self,y,y_pred):
        return y_pred-y
    
    def forward_pass(self,x,y,weights,bias):
        cost=0
        self.outputs=[]
        for i in range(len(weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            self.outputs.append(x) #append without adding ones array
            z=np.dot(np.append(ones_array,x,axis=1),weights[i]+bias[i])
            if i==len(weights)-1:
                x=self.softmax(z)
            else:
                x=self.activation(self.activations[i],z)
        self.outputs.append(x)
        self.y_pred=x
        
        temp=-self.max_log_likelihood(self.y_pred,y)
        cost=np.mean(np.sum(temp,axis=1))
        return cost
    
    
    def backward_pass(self,y,lr,beta=0.9,momentum=False):
        for i in range(len(self.weights)-1,-1,-1):
            ones_array=np.ones(len(self.outputs[i])).reshape(len(self.outputs[i]),1)
            if i==len(self.weights)-1:
                prev_term=self.delta_mll(y,self.y_pred)  
                # derivatives follow specific order,last three terms added new,rest from previous term  
                self.derivatives[i]=np.dot(prev_term.T,np.append(ones_array,self.outputs[i],axis=1))   
            else:
                prev_term=np.dot(prev_term,self.weights[i+1][1:].T)*self.activation(self.activations[i],self.outputs[i+1],derivative=True)
                self.derivatives[i]=np.dot(prev_term.T,np.append(ones_array,self.outputs[i],axis=1))
            if momentum:
                self.delta_weights[i]=beta*self.delta_weights[i]-lr*((self.derivatives[i].T)/len(y))
                self.delta_bias[i]=beta*self.delta_bias[i]-lr*((self.derivatives[i].T)/len(y))
                self.weights[i]=self.weights[i]+self.delta_weights[i]
                self.bias[i]=self.bias[i]+self.delta_bias[i]
            else:
                self.weights[i]=self.weights[i]-lr*((self.derivatives[i].T)/len(y))
                self.bias[i]=self.bias[i]-lr*((self.derivatives[i].T)/len(y))
    
    def train(self,batches,beta,lr=1e-3,epoch=10):
        for epochs in range(epoch):
            samples=len(self.x)
            c=0
            for i in range(batches):
              x_batch=self.x[int((samples/batches)*i):int((samples/batches)*(i+1))]
              y_batch=self.y.loc[int((samples/batches)*i):int((samples/batches)*(i+1))-1]
              
              c=self.forward_pass(x_batch,y_batch,self.weights,self.bias)
              self.backward_pass(y_batch,lr,beta,momentum=True)
            print(epochs,c/batches)
    
    def predict(self,x):
        x=x/255
        for i in range(len(self.weights)):
            samples=len(x)
            ones_array=np.ones(samples).reshape(samples,1)
            z=np.dot(np.append(ones_array,x,axis=1),self.weights[i]+self.bias[i])
            if i==len(self.weights)-1:
                x=self.softmax(z)
            else:
                x=self.activation(self.activations[i],z)
        return np.argmax(x,axis=1)

In [40]:
neuron = NeuralNetworkMomentum(x_train, Labels)
layer1 = Layer(100, 'sigmoid')
layer2 = Layer(50, 'tanh')
neuron.connect(x_train, layer1)
neuron.connect(layer1, layer2)
neuron.connect(layer2, Labels)
neuron.train(batches=500, lr=0.1, epoch=20, beta=0.5)

0 0.002017783444407748
1 0.0012781967915228708
2 0.0012555595624274309
3 0.0011586001983104682
4 0.00037248889935154644
5 0.00028085807989264777
6 7.78206282498589e-05
7 5.853374177009288e-05
8 4.990565018824331e-05
9 0.0001378387654306032
10 0.00010703481169217598
11 1.3948708837239218e-05
12 4.2646829030458464e-05
13 4.089017302283832e-05
14 5.529278689862014e-06
15 6.611416310459818e-06
16 4.3141944244476665e-06
17 1.0121940113461146e-05
18 1.0059310850684675e-05
19 8.969774898630038e-06


In [41]:
pred = neuron.predict(x_test)
np.bincount(neuron.predict(x_test)), np.bincount(y_test)

(array([260, 266, 256, 276, 264, 223, 235, 268, 214, 238]),
 array([256, 268, 262, 271, 266, 232, 231, 276, 206, 232]))

In [42]:
print(f"accuracy is {np.bincount(np.abs(y_test-pred))[0]*100/len(y_test)} %")

accuracy is 93.36 %
